In [439]:
!pip install pyinterval

In [440]:
from interval import interval
import pandas as pd

In [441]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:4.3f}, {interv[0].sup:4.3f}]"

In [442]:
def calculate_volterra1(f, expected, k, y0, a, b, c, d, p):
    h = (b - a) / p
    x = []
    y = []
    exp = []
    exp.append(interval[expected(a),expected(a)])
    for i in range(0, p):
        xi = a + i*h
        xinext = xi + h
        x.append(interval[xi, xinext])
        sum = 0
        for j in range(0, i):
            sum += k(x[i], x[j], y0[j])
        y.append(f(x[i]) + h*sum + interval[0,h]*k(x[i], x[i], y0[i]))
        exp.append(expected(xi))
    u = []
    u.append(y0[0])
    for i in range(0,p-1):
        u.append(y[i]&y[i+1])
    return x, y, u, exp

In [443]:
a = 0
b = 1/4
c = 0
d = 2
p = 3
n = 10
y0 = []
for i in range(p):
    y0.append(interval[c,d])
func = lambda x: 1
k = lambda x,s,y : y**2
kdf = lambda x,s,y : 2*y
expected = lambda x: 1/(1-x)

In [444]:
#tbl1 = pd.DataFrame(columns=['xi', 'yi', 'm(yi)', 'expected', 'e'])

for iteration in range(n):
    print(f"n={iteration+1}#########################################################################")
    x, y, u, exp = calculate_volterra1(func, expected, k, y0, a, b, c, d, p)
    print("xi\t\tyi\t\tui\t\tm(ui)\treal\tdifference")
    for i in range(p):
        mid_point=calculate_middle_point(u[i])
        real = expected(x[i][0].inf)
        print(f"{to_string(x[i])}\t{to_string(y[i])}\t{to_string(u[i])}\t{mid_point:4.4}\t{real:4.4}\t{abs(mid_point-real)}")
    y0 = y 

n=1#########################################################################
xi		yi		ui		m(ui)	real	difference
[0.000, 0.083]	[1.000, 1.333]	[0.000, 2.000]	 1.0	 1.0	0.0
[0.083, 0.167]	[1.000, 1.667]	[1.000, 1.333]	1.167	1.091	0.07575757575757569
[0.167, 0.250]	[1.000, 2.000]	[1.000, 1.667]	1.333	 1.2	0.13333333333333353
n=2#########################################################################
xi		yi		ui		m(ui)	real	difference
[0.000, 0.083]	[1.000, 1.148]	[1.000, 1.333]	1.167	 1.0	0.16666666666666674
[0.083, 0.167]	[1.083, 1.380]	[1.083, 1.148]	1.116	1.091	0.02483164983164987
[0.167, 0.250]	[1.167, 1.713]	[1.167, 1.380]	1.273	 1.2	0.0731481481481484
n=3#########################################################################
xi		yi		ui		m(ui)	real	difference
[0.000, 0.083]	[1.000, 1.110]	[1.000, 1.148]	1.074	 1.0	0.07407407407407418
[0.083, 0.167]	[1.083, 1.268]	[1.083, 1.110]	1.097	1.091	0.005684416178243223
[0.167, 0.250]	[1.181, 1.513]	[1.181, 1.268]	1.225	 1.2	0.024801383173296